# Higher-order functions

Higher-order functions (HOFs) are functions which receive or return other functions. HOFs show that functions can be regarded as _values_, much in the same way than integers, booleans, product values, sum values, etc. We will see that higher-order functions are essential modularity devices, and we will introduce the most common higher-order functions that operate over many different data structures.

## Functions as values

As we saw, functions are represented through methods in Scala (as in any other object-oriented language). But methods themselves can't be passed around and be returned by invocations. Therefore, the first thing to do in order to create HOFs in Scala is finding a way to reify methods. For instance, let's consider the following functions:

In [ ]:
def addOneM(number: Int): Int = 
    number + 1

def substractOneM(number: Int): Int = 
    number - 1 

We want to implement a function that receives an integer-to-integer function, such as `addOneM`and `substractOneM`, and call this function over a given number. We may want to write something like this:

where the first argument `int2int` attempt to represent any function that receives an integer and returns another integer. 

This code is not legal in Scala, but we can create a new class whose only method is the function that we want to pass around:

Now, we can implement the `call` HOF as follows: 

In [ ]:
/*def call(int2int(i: Int): Int, number: Int): Int = 
    ??? 
*/

In order to use this HOF with the `addOneM` and `substractOneM` functions, we must create reified versions for them: 

In [ ]:
/*
def addOneM(number: Int): Int = 
    number + 1

def substractOneM(number: Int): Int = 
    number - 1 
*/

We call the `addOneV` and `substractOneV` function-values, i.e. functions represented as values. Now, we can use the `call` HOF as follows:

### Standard functions in Scala

The Scala programming language offers many facilities to work with functions as values. First, the standard library provides the following _generic_ types [`Function1`](https://www.scala-lang.org/api/current/scala/Function1.html), [`Function2`](https://www.scala-lang.org/api/current/scala/Function2.html), ...:

In [ ]:
/*
abstract class FunctionInt2Int{
    def apply(number: Int): Int
}
*/

Using these standard classes, we can create the `addOneV` function-value in a similar way than before: 

In [ ]:
/*
val addOneV: FunctionInt2Int = new FunctionInt2Int{
    def apply(number: Int): Int = 
        number + 1
}
*/

But we can do it more easily, since Scala also provides special syntax to declare function types and create functions (so-called _lambda expressions_):

And we can also profit from type inference:

Using these syntactic facilities we can write the `call` HOF more easily: 

which we can use as follows:

We can even pass function-methods that are converted on the fly to function-values (this is the so-called _eta-expansion_):

### Currying

We can use a similar syntax for functions of two arguments. So, instead of writing the more verbose:

In [ ]:
// sum/2


we can create a lambda expressions for a `Function2` value in the following way:

or, exploiting type inference:

However, `Function2`, `Function3`, ... classes are not extrictly necessary, and sometimes we can get along with `Function1`. But, how can we create a function of two arguments with `Function1` alone? The trick is the following:

Note that brackets in `Int => (Int => Int)` are used for clarity, but are not needed. Basically, we created a function of one argument that returns another function of one argument. So, the expression: 

returns a function that can be applied again:

We can apply this strategy to functions of any number of arguments. This is called _currying_.

### Improved unit testing

In [ ]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

Using higher-order functions we can create test catalogues which are parameterised by the function to be tested. For instance, let's consider the following two alternative implementations:

In [ ]:
// Recursively

def sumR(list: List[Int]): Int = 
    list match {
        case Nil => 0 : Int
        case head :: tail => head + sumR(tail) : Int 
    }

In [ ]:
// With tail-recursion

def sumTR(list: List[Int]): Int = {

    def sumAux(acc: Int, list: List[Int]): Int = 
        list match {
            case Nil => acc : Int
            case head :: tail => sumAux(head + acc, tail) : Int 
        }
    
    sumAux(0, list)
}

Instead of creating ad-hoc test catalogues for each alternative, we can create a single one that receives the function to be tested as argument as follows:

In [ ]:
/*
object TestSumR extends FlatSpec with Matchers{
    "length" should "work" in {
        sumR(List()) shouldBe 0 
        sumR(List(1)) shouldBe 1 
        sumR(List(1,2,3,4)) shouldBe 10
    }
}
*/


Now, we can test the `sumR` and `sumTR` functions by reusing the same test catalogue:

## Functions compose

We can create new functions by composing other functions whose signatures match. This is great from a modularity perspective. For instance, the following function is implemented in a non-modular way:

In [ ]:
// isEvenLength


This function is somehow the combination of two more basic functions `length` and `isEven`:

but this is not reflected in the current implementation. How can we redefine the function `isEvenLength` from the functions `length` and `isEven`? We can use a HOF which is able to compose functions:

Then, we can redefine `isEvenLength` in a modular way from the `length` and `isEven` building blocks:

In [ ]:
/*
def isEvenLength: String => Boolean = 
    (s: String) => s.length % 2 == 0
    */

The HOF `compose` is actually defined by `Function1`: 

or using infix notation:

Note that a similar function to `compose`, called `andThen`, is also available: 

## `FoldRight`: divide and conquer

HOFs shine when the time comes to break monoliths. For instance, let's consider the following two functions:

In [ ]:
def sum(list: List[Int]): Int = 
    list match {
        case Nil => 0
        case head :: tail => head + sum(tail)
    }

In [ ]:
def multiply(list: List[Int]): Int = 
    ???

These functions clearly share a common logic; their only differences are the value which is returned when the list is empty, and the function used to combine numbers (`+` and `*`, respectively). We can abstract away these differences and arrive to a more generic function which encodes that common logic:

In [ ]:
/*
def sum(list: List[Int]): Int = 
    list match {
        case Nil => 0
        case head :: tail => head + sum(tail)
    }
*/

which allows us to re-define in a modular way the `sum` and `multiply` functions:

In [ ]:
/*
def sum(list: List[Int]): Int = 
    list match {
        case Nil => 0
        case head :: tail => head + sum(tail)
    }
*/

In [ ]:
/*
def multiply(list: List[Int]): Int = 
    list match {
        case Nil => 1
        case head :: tail => head * multiply(tail)
    }
*/

But we don't need to constrain ourselves to integers. In its generic version, the `combine` function is actually the `foldRight` higher-order function (for `List`s):

In [ ]:
/*
def combine(list: List[Int])(nil: Int, cons: (Int, Int) => Int): Int = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, combine(tail)(nil, cons))
    }
*/

Graphically, we can explain the behaviour of `foldRight` as follows: 

![with elements](images/foldRight.1.svg)
(all images credit to [Scala Visual Reference](https://superruzafa.github.io/visual-scala-reference/))

We can understand `foldRight` as an implementation of the divide-and-conquer design pattern: first, divide your problems in subproblems; second, solve these sub-problems; last, compose their solutions to obtain the solution to the overall problem. If the sub-problems are simple enough they can be solved directly; otherwise, they are solved recursively. In the case of lists:
- The problem is obtaining a value of type `B` for a given list.
- The only sub-problem corresponds to the tail of that list. 
- The arguments of the `foldRight` function tell us how to obtain the solution for the empty list (the atomic problem which can not be further decomposed), and how to obtain the solution from the solution to the sub-problem.

From this perspective, the implementation of `sum` and `multiply` is exactly the same as before when we used the function `combine`, although we can explain them differently:

In [ ]:
/*def sum(list: List[Int]): Int = 
    combine(list)(0, (a, b) => a + b)
*/

In [ ]:
/*
def multiply(list: List[Int]): Int = 
    combine(list)(1, _ * _)
*/

#### Improved type-inference

Technically, you may have noticed that we have to give extra type information in the invocations to `foldRight`. In fact, the following code doesn't compile. Check it yourself!

In [ ]:
/*
def multiply(list: List[Int]): Int = 
    foldRight(list)(1, (a, b) => a * b)
*/

In order to help the Scala compiler to infer the type parameters of the `foldRight` function, we need to change its signature a little bit: 

In [ ]:
/*
def foldRight[A, B](list: List[A])(nil: B, cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil, cons))
    }
*/

By splitting the second parameter list, we allow the Scala compiler to infer the type of `B`, before it analyses the type of the `cons` argument. Now this works:

#### HOFs in the Scala API

The `foldRight` HOF, as all the HOFs that we will explain in this notebook, are actually part of the standard Collections library of Scala. Typically, they are implemented as member methods of the corresponding collection class. For lists, the [Scala API](https://www.scala-lang.org/api/2.13.1/scala/collection/immutable/List.html) offers something like this:

So, invocations of the standard `foldRight` do not receive the list as argument, but are common method invocations on some `List` instance:

Normally, we will use standard HOFs instead of our home-made versions, once they are explained. 

## The Hall of Fame of HOFs

Besides `compose`, `andThen` and `foldRight`, there are other famous higher-order functions which work great as modularity devices: 

In [ ]:
// foldLeft
// filter
// map
// flatMap

But most of these functions do not only work for `List`, they work also for `Option`, `Either[A, ?]`, and many other data structures. For instance:

In [ ]:
// fold
// filter
// map
// flatMap

In [ ]:
// fold
// filter
// map
// flatMap

The `fold` function is the catamorphism for `Option` and `Either[A, ?]`, in the same way than `foldRight` is the catamorphism for lists (we will see catamorphisms later on). 

## FoldLeft: a better loop

The `foldLeft` HOF is the functional way to implement common imperative algorithms. In the following implementation, its signature declares the initial value of a mutable variable, and the update function executed in every step of the loop:

In [ ]:
def foldLeft[A, B](list: List[A])(initial: B)(update: (B, A) => B): B = 
    ???

Similarly, the `foldLeft` function is typically used where a tail-recursive function is needed. Its recursive implementation is naturally tail-recursive:

In [ ]:
@annotation.tailrec
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ???

Using `foldLeft` we can implement functions at a higher-level of abstraction, i.e. using constructs which are nearer to the problem-domain, without caring about mutable variables, tail-safe recursion, etc. Its use also leads to very concise (and readable!) implementations. For instance, these are (almost) one-liner implementations of some of the functions of the last notebook on recursive functions:

In [ ]:
def length[A](list: List[A]): Int = 
    ???

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [ ]:
def reverse[A](list: List[A]): List[A] = 
    ???

But we may also use the `foldRight` function to implement them:

In [ ]:
def length[A](list: List[A]): Int = 
    ???

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [ ]:
def reverse[A](list: List[A]): List[A] = 
    ???

Functionally, both versions are equivalent, but the ones using `foldLeft` are more efficient. In particular, the `foldRight` version of `reverse` has cuadratic complexity. 

#### Implementing `foldRight` with `foldLeft`

The implementation that we gave for `foldRight` was not tail-recursive, so this will blow up the stack:

In order to obtain a better implementation, we may first reverse the list and use `foldLeft` as follows: 

In [ ]:
def foldRightTR[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    ???

This implementation works right with big lists:

In [ ]:
// foldRightTR[Int, Int](List.fill(100000)(1))(0)((_,_)=> 0)

## The `map` HOF

The `map` function is one of the essential HOFs. Its purpose is applying a function to the elements of a data structure, in such a way that the relationships between these elements do not change. The only thing that must be modified is the _content_ of the data structure, not its _shape_. This condition is expressed in the following equations:

1. `map(list)(identity) == list` for all `list: List[A]`
2. `map(map(list)(f))(g) == map(list)(g compose f)` for all `list: List[A]`, `f: A => B`, `g: B => C`

![map](images/map.svg)

The implementation for lists can be done as follows:

In [ ]:
def map[A, B](list: List[A])(f: A => B): List[B] = 
    ???

The `map` function is polymorphic in `A` and `B`, but we can't pass generic functions (aka polymorphic values) as parameters using `FunctionN`classes (these classes only wrap functions from specific types to specific result types). We may create polymorphic versions of `FunctionN` clases, but in order to keep things simple, we will define the test catalogue for `map` using a specific signature chosen at random (any other may serve as well):

In [ ]:
class TestMap(
    map: List[Int] => (Int => Boolean) => List[Boolean]
) extends FlatSpec with Matchers{
    "mapping an empty list" should "return an empty list" in {
        
    }
    
    "mapping an non-empty list" should "only change its content" in {
        
    }
}

To run the tests we will pass the `map` function for `Ìnt` and `Boolean` types:

although it's not necessary to write the types explicitly (Scala will infer the types for us):

The implementation of `map` is really close to the implementation of `foldRight`. Indeed, we can give a more modular implementation using this more basic HOF: 

In [ ]:
/*
def foldRight[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil)(cons))
    }
*/

In [ ]:
run(new TestMap(mapFR))

### `map` for `Option`

The implementation of the `map` function given above works over lists, but we can also map optional values and many other data structures. The implementation for the data structure `Option` must satisfy the following test:

In [ ]:
class TestMapOption(
    map: Option[Int] => (Int => Boolean) => Option[Boolean]
) extends FlatSpec with Matchers{
    
    "mapping the `None` value" should "return `None`" in {
        
    }
    
    "mapping a `Some` value" should "only change its content" in {
        
    }    
}

The implementation is simple:

In [ ]:
def mapOpt[A, B](maybeA: Option[A])(f: A => B): Option[B] = 
    ???

In [ ]:
run(new TestMapOption(mapOpt))

## Filtering elements

Unlike `map`, the `filter` HOF allows us to change the shape of the data structure, removing those elements that do not satisfy a given predicate. 

![filter](images/filter.svg)

In [ ]:
def filterR[A](list: List[A])(predicate: A => Boolean): List[A] = 
    ???

In [ ]:
class TestFilterList(
    filter: List[Int] => (Int => Boolean) => List[Int]
) extends FlatSpec with Matchers{
    
    "filter" should "work" in {
    }
}

In [ ]:
run(new TestFilterList(filter))

Using `foldRight` we can get a more modular implementation: 

In [ ]:
def filterFR[A](list: List[A])(predicate: A => Boolean): List[A] = 
    ???

In [ ]:
run(new TestFilterList(filterFR))

### Filtering optional values

We only have a possible value, so implementing filtering is easy in this case:

In [ ]:
class TestFilterOption(
    filter: Option[Int] => (Int => Boolean) => Option[Int]
) extends FlatSpec with Matchers{
    
    "filter" should "work" in {
    }
}

In [ ]:
def filter[A](maybeA: Option[A])(predicate: A => Boolean): Option[A] = 
    ???

In [ ]:
run(new TestFilterOption(filter))

## FlatMapping data structures

Let's consider a paragraph represented as a list of sentences, where each sentence is in turn modelled as a string made of _words_ separated by blank spaces. We want to obtain a list of all the words in each sentence. 

In [ ]:
class TestWords(
    words: List[String] => List[String]
) extends FlatSpec with Matchers{
            
    val paragraph1 = List(
        "En un lugar",
        "de la Mancha", 
        "de cuyo nombre no",
        "quiero acordarme")
    
    "words" should "work" in {
    
    }
}

We may try to map the paragraph with a function that `split`s  each sentence into the words they are made of:

but then we obtain a _list of lists_ of strings, not a list of plain strings. The solution is not far though: we have just to concatenate all the lists and we obtain what we need. The function `flatten` performs this concatenation:

The function `flatten` is actually provided by the Scala API. So, the implementation of the `words` function is as follows:

In [ ]:
def words(paragraph: List[String]): List[String] = 
    ???

In [ ]:
run(new TestWords(words))

This combination of the HOF `map` and the function `flatten` is so common, that it has been given a proper name: `flatMap`. 

![filter](images/flatMap.svg)

Using `flatMap` the word function is implemented even more easily:

In [ ]:
def words(paragraph: List[String]): List[String] = 
    ???

## All together: a modularity problem

As before, we start from a list of sentences, but now we want to take care of the possible extra spaces between each word. Also, we want to obtain not the words themselves but their lengths. So, the specification of our problem is a function `lengths`:

In [ ]:
def lengths(paragraph: List[String]): List[Int] = ???

such that: 

In [ ]:
class TestLengths(
    lengths: List[String] => List[Int]
) extends FlatSpec with Matchers{
            
    val paragraph1 = List(
        "En un  lugar",
        "de  la Mancha ", 
        "de cuyo nombre no",
        "quiero        acordarme")
    
    "lengths" should "work" in {
    }
}

We may try to implement this function imperatively using mutable variables:

In [ ]:
def lengthsM(paragraph: List[String]): List[Int] = 
    ???

and this works, of course: 

In [ ]:
run(new TestLengths(lengthsM))

but can we do it better? Yes, we can! Using HOFs we can get a more concise, understandable and reliable version, with a great level of reuse!

In [ ]:
def lengthsHOF(paragraph: List[String]): List[Int] = 
    ???

In [ ]:
run(new TestLengths(lengthsHOF))

This solution contrasts with the mutable version in several respects:
- It's more **modular**, i.e. it's made of coarse-grained components: the HOFs `flatMap`, `filter` and `map`. The mutable version builds instead upon fine-grained components: `var`s, and `for` loops.
- The HOF components `flatMap`, etc., are actually very generic and domain-independent, and are typically part of standard libraries, so the level of **reuse** and **reliability** of the HOF-based solution is very high.
- The HOF-based solution is more **understandable**: it models the solution to the problem in terms of standard _transformations_ which are composed together using plain function composition. Moreover, these transformations are also at the right level of abstraction, i.e. it's natural to specify the solution to the problem in terms of flatMap, filter and map.

### For-comprehensions

The combination of `map`, `flatMap` and `filter` HOFs is so common, that Scala has a special syntax for them: for-comprehensions.

In [ ]:
def lengthsFC(paragraph: List[String]): List[Int] = 
    ???

In [ ]:
run(new TestLengths(lengthsFC))